In [ ]:
# yfinance, getFamaFrenchFactors 패키지 설치
!pip install yfinance --quiet
!pip install statsmodels --quiet
!pip install getFamaFrenchFactors --quiet

In [1]:
# 패키지 임포트: pandas, statsmodels, yfinance, getFamaFrenchFactors
import pandas as pd
import yfinance as yf
import statsmodels.api as sm
import getFamaFrenchFactors as gff

###  [`statsmodels`](https://www.statsmodels.org/dev/examples/index.html)

> **R에서만 가능했던 분석 기능들을 그대로 파이썬에서 이용가능**
> - 예제 데이터셋
> - 검정 및 추정
> - 선형/로지스틱/강건/일반화 회귀분석
> - 혼합효과모형
> - 이산종속변수
> - 요인분석
> - 생존분석
> - 시계열 분석
> - 상태공간모형


### [yfinance](https://pypi.org/project/yfinance/)

>[야후 파이낸스(Yahoo! Finance)](https://finance.yahoo.com/)를 통해 금융 데이터를 액세스 가능한 오픈 소스 라이브러리
>주식, 채권, 통화 및 암호화폐에 대한 광범위한 시장 데이터를 제공
>* 현재 재무 데이터 접근 안 되고, 가격 데이터만 불러와짐

### 애플 과거 주가 데이터 다운로드

In [2]:
# 주가 데이터 다운로드
# 애플 주식 티커
ticker = 'AAPL'

# 데이터 시작일과 종료일
start = '2000-01-01'
end = '2023-05-08'

# 데이터 다운로드
stock_data = yf.download(ticker, start, end)

# 주가 수익률 계산
stock_returns = stock_data['Adj Close'].resample('M').last().pct_change().dropna()
stock_returns.name = 'Month_Rtn'
stock_returns.index = pd.to_datetime(stock_returns.index).date

[*********************100%***********************]  1 of 1 completed


In [3]:
stock_returns

2000-02-29    0.104820
2000-03-31    0.184842
2000-04-30   -0.086517
2000-05-31   -0.322922
2000-06-30    0.247024
                ...   
2023-01-31    0.110521
2023-02-28    0.023183
2023-03-31    0.118649
2023-04-30    0.028987
2023-05-31    0.022926
Name: Month_Rtn, Length: 280, dtype: float64

In [4]:
train_data = stock_returns.iloc[:200]
test_data = stock_returns.iloc[200:]
train_data, test_data

(2000-02-29    0.104820
 2000-03-31    0.184842
 2000-04-30   -0.086517
 2000-05-31   -0.322922
 2000-06-30    0.247024
                 ...   
 2016-05-31    0.071773
 2016-06-30   -0.042660
 2016-07-31    0.090062
 2016-08-31    0.023652
 2016-09-30    0.065505
 Name: Month_Rtn, Length: 200, dtype: float64,
 2016-10-31    0.004334
 2016-11-30   -0.021601
 2016-12-31    0.047955
 2017-01-31    0.047747
 2017-02-28    0.133778
                 ...   
 2023-01-31    0.110521
 2023-02-28    0.023183
 2023-03-31    0.118649
 2023-04-30    0.028987
 2023-05-31    0.022926
 Name: Month_Rtn, Length: 80, dtype: float64)

### 파마-프렌치 3 팩터 데이터 다운로드

In [5]:
# 파마-프렌치 3 팩터 데이터 다운로드
df_ff3_monthly = gff.famaFrench3Factor(frequency='m')
df_ff3_monthly.rename(columns={'date_ff_factors': 'Date'}, inplace=True)
df_ff3_monthly.set_index('Date', inplace=True)
df_ff3_monthly.index = pd.to_datetime(df_ff3_monthly.index).date

# 팩터 데이터와 주가 데이터 병합
ff_data = pd.concat([df_ff3_monthly, stock_returns], axis=1, join='inner')
ff_data_train = pd.concat([df_ff3_monthly, train_data], axis=1, join='inner')
ff_data_test = pd.concat([df_ff3_monthly, test_data], axis=1, join='inner')

In [6]:
ff_data

,Mkt-RF,SMB,HML,RF,Month_Rtn
2000-02-29,0.0245,0.2142,-0.0970,0.0043,0.104820
2000-03-31,0.0520,-0.1723,0.0817,0.0047,0.184842
2000-04-30,-0.0640,-0.0668,0.0726,0.0046,-0.086517
2000-05-31,-0.0442,-0.0609,0.0481,0.0050,-0.322922
2000-06-30,0.0464,0.1285,-0.0843,0.0040,0.247024
...,...,...,...,...,...
2023-01-31,0.0665,0.0502,-0.0405,0.0035,0.110521
2023-02-28,-0.0258,0.0121,-0.0078,0.0034,0.023183
2023-03-31,0.0251,-0.0559,-0.0901,0.0036,0.118649
2023-04-30,0.0061,-0.0334,-0.0003,0.0035,0.028987


In [7]:
ff_data_train

,Mkt-RF,SMB,HML,RF,Month_Rtn
2000-02-29,0.0245,0.2142,-0.0970,0.0043,0.104820
2000-03-31,0.0520,-0.1723,0.0817,0.0047,0.184842
2000-04-30,-0.0640,-0.0668,0.0726,0.0046,-0.086517
2000-05-31,-0.0442,-0.0609,0.0481,0.0050,-0.322922
2000-06-30,0.0464,0.1285,-0.0843,0.0040,0.247024
...,...,...,...,...,...
2016-05-31,0.0178,-0.0018,-0.0166,0.0001,0.071773
2016-06-30,-0.0005,0.0060,-0.0148,0.0002,-0.042660
2016-07-31,0.0395,0.0251,-0.0127,0.0002,0.090062
2016-08-31,0.0049,0.0118,0.0313,0.0002,0.023652


In [8]:
ff_data_test

,Mkt-RF,SMB,HML,RF,Month_Rtn
2016-10-31,-0.0202,-0.0442,0.0412,0.0002,0.004334
2016-11-30,0.0486,0.0567,0.0819,0.0001,-0.021601
2016-12-31,0.0181,0.0008,0.0356,0.0003,0.047955
2017-01-31,0.0194,-0.0114,-0.0276,0.0004,0.047747
2017-02-28,0.0357,-0.0202,-0.0168,0.0004,0.133778
...,...,...,...,...,...
2023-01-31,0.0665,0.0502,-0.0405,0.0035,0.110521
2023-02-28,-0.0258,0.0121,-0.0078,0.0034,0.023183
2023-03-31,0.0251,-0.0559,-0.0901,0.0036,0.118649
2023-04-30,0.0061,-0.0334,-0.0003,0.0035,0.028987


In [9]:
# ff_data 데이터프레임 칼럼 이름 변경
ff_data.columns = ['beta', 'size', 'value', 'rf', 'ret']
ff_data_train.columns = ['beta', 'size', 'value', 'rf', 'ret']
ff_data_test.columns = ['beta', 'size', 'value', 'rf', 'ret']

# 결과
ff_data

,beta,size,value,rf,ret
2000-02-29,0.0245,0.2142,-0.0970,0.0043,0.104820
2000-03-31,0.0520,-0.1723,0.0817,0.0047,0.184842
2000-04-30,-0.0640,-0.0668,0.0726,0.0046,-0.086517
2000-05-31,-0.0442,-0.0609,0.0481,0.0050,-0.322922
2000-06-30,0.0464,0.1285,-0.0843,0.0040,0.247024
...,...,...,...,...,...
2023-01-31,0.0665,0.0502,-0.0405,0.0035,0.110521
2023-02-28,-0.0258,0.0121,-0.0078,0.0034,0.023183
2023-03-31,0.0251,-0.0559,-0.0901,0.0036,0.118649
2023-04-30,0.0061,-0.0334,-0.0003,0.0035,0.028987


### 파마-프렌치 3 팩터 모델로 주가 선형 회귀 분석

In [10]:
# 회귀분석 설명변수: 베타, 사이즈, 밸류
X = ff_data[['beta', 'size', 'value']]

# 회귀분석 종속변수: 애플 주식의 무위험 초과 수익률
y = ff_data['ret'] - ff_data['rf']

# 데이터를 회귀분석 모델에 피팅시키기
ff_model = sm.OLS(y, X).fit()

# 회귀분석 결과 출력
print(ff_model.summary())

# 베타 계수 할당
b1, b2, b3 = ff_model.params

                                 OLS Regression Results                                
Dep. Variable:                      y   R-squared (uncentered):                   0.374
Model:                            OLS   Adj. R-squared (uncentered):              0.367
Method:                 Least Squares   F-statistic:                              55.11
Date:                Sat, 12 Aug 2023   Prob (F-statistic):                    5.76e-28
Time:                        14:12:46   Log-Likelihood:                          271.73
No. Observations:                 280   AIC:                                     -537.5
Df Residuals:                     277   BIC:                                     -526.5
Df Model:                           3                                                  
Covariance Type:            nonrobust                                                  
                 coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------

In [11]:
b1, b2, b3

(1.3768127624954283, 0.11012202369891347, -0.7436167367604901)

In [ ]:
# 무위험수익률
rf = ff_data_train['rf'].mean()

# 시장 위험 프리미엄
market_premium = ff_data_train['beta'].mean()

# 사이즈 프리미엄
size_premium = ff_data_train['size'].mean()

# 밸류 프리미엄
value_premium = ff_data_train['value'].mean()

# 팩터를 활용한 기대수익률 계산
# 월간 기대수익률
expected_monthly_return = rf + b1 * market_premium + b2 * size_premium + b3 * value_premium

# 연간 기대수익률
expected_yearly_return = expected_monthly_return * 12

# 기대수익률 출력
print('Expected Yearly Return: ' + str(expected_yearly_return))

In [ ]:
# 회귀분석 설명변수: 베타, 사이즈, 밸류
X = ff_data_train[['beta', 'size', 'value']]

# 회귀분석 종속변수: 애플 주식의 무위험 초과 수익률
y = ff_data_train['ret'] - ff_data_train['rf']

# 데이터를 회귀분석 모델에 피팅시키기
ff_model = sm.OLS(y, X).fit()

# 회귀분석 결과 출력
print(ff_model.summary())

# 베타 계수 할당
b1, b2, b3 = ff_model.params

In [ ]:
# 무위험수익률
rf = ff_data_test['rf']

# 시장 위험 프리미엄
market_premium = ff_data_test['beta']

# 사이즈 프리미엄
size_premium = ff_data_test['size']

# 밸류 프리미엄
value_premium = ff_data_test['value']

# 팩터를 활용한 기대수익률 계산
# 월간 기대수익률
expected_monthly_return = rf + b1 * market_premium + b2 * size_premium + b3 * value_premium

In [ ]:
ret_data = pd.concat([expected_monthly_return, test_data], axis=1, join='inner')
ret_data.columns=['Expected', 'Real']
ret_data

In [ ]:
import matplotlib.pyplot as plt
expected_monthly_return.cumsum().plot(label='expected')
test_data.cumsum().plot(label='real')
plt.legend()

### 시간 가변적 베타 (Time-Varying Beta)


In [ ]:
# RollingOLS 임포트
from statsmodels.regression.rolling import RollingOLS

In [ ]:
ff_data

In [ ]:
# 3-팩터 롤링 베타 추출
betas = RollingOLS.from_formula('ret ~ beta + size + value', data=ff_data, window=36).fit().params

In [ ]:
betas

In [ ]:
b0 = betas['Intercept']
b1 = betas['beta']
b2 = betas['size']
b3 = betas['value']

In [ ]:
# 무위험수익률
rf = ff_data['rf']

# 시장 위험 프리미엄
market_premium = ff_data['beta']

# 사이즈 프리미엄
size_premium = ff_data['size']

# 밸류 프리미엄
value_premium = ff_data['value']

# 팩터를 활용한 기대수익률 계산
# 월간 기대수익률
expected_monthly_return = b1 * market_premium + b2 * size_premium + b3 * value_premium + b0 - rf

In [ ]:
ret_data = pd.concat([expected_monthly_return, test_data], axis=1, join='inner')
ret_data.columns=['Expected', 'Real']
ret_data

In [ ]:
expected_monthly_return.cumsum().plot(label='Expected')
stock_returns.cumsum().plot(label='Real')
plt.legend()

In [ ]:
# 팩터 계수 그래프
betas.dropna().plot(figsize=(10, 6))